In [8]:
import os
data_dir = 'C:/Users/manci/Desktop/data/PythonML-main/PythonML-main/chapter06/data'
input_dir = os.path.join(data_dir, '0_input')
output_dir = os.path.join(data_dir, '10_output')
master_dir = os.path.join(data_dir, '99_master')
print(input_dir)

C:/Users/manci/Desktop/data/PythonML-main/PythonML-main/chapter06/data\0_input


In [9]:
os.makedirs(input_dir, exist_ok = True)
os.makedirs(output_dir, exist_ok = True)
os.makedirs(master_dir, exist_ok = True)

In [10]:
import glob
tbl_order_file = os.path.join(input_dir, 'tbl_order_*.csv')
tbl_order_paths = glob.glob(tbl_order_file)
tbl_order_paths

['C:/Users/manci/Desktop/data/PythonML-main/PythonML-main/chapter06/data\\0_input\\tbl_order_202004.csv',
 'C:/Users/manci/Desktop/data/PythonML-main/PythonML-main/chapter06/data\\0_input\\tbl_order_202005.csv',
 'C:/Users/manci/Desktop/data/PythonML-main/PythonML-main/chapter06/data\\0_input\\tbl_order_202006.csv',
 'C:/Users/manci/Desktop/data/PythonML-main/PythonML-main/chapter06/data\\0_input\\tbl_order_202007.csv',
 'C:/Users/manci/Desktop/data/PythonML-main/PythonML-main/chapter06/data\\0_input\\tbl_order_202008.csv',
 'C:/Users/manci/Desktop/data/PythonML-main/PythonML-main/chapter06/data\\0_input\\tbl_order_202009.csv',
 'C:/Users/manci/Desktop/data/PythonML-main/PythonML-main/chapter06/data\\0_input\\tbl_order_202010.csv',
 'C:/Users/manci/Desktop/data/PythonML-main/PythonML-main/chapter06/data\\0_input\\tbl_order_202011.csv',
 'C:/Users/manci/Desktop/data/PythonML-main/PythonML-main/chapter06/data\\0_input\\tbl_order_202012.csv',
 'C:/Users/manci/Desktop/data/PythonML-main/Py

In [11]:
import pandas as pd

m_area_file = 'C:/Users/manci/Desktop/data/PythonML-main/PythonML-main/chapter06/data/m_area.csv'
m_store_file = 'C:/Users/manci/Desktop/data/PythonML-main/PythonML-main/chapter06/data/m_store.csv'
m_area = pd.read_csv(os.path.join(master_dir, m_area_file))
m_store = pd.read_csv(os.path.join(master_dir, m_store_file))
m_area.head(3)

,area_cd,wide_area,narrow_area
0,SL,서울,서울
1,BS,부산,부산
2,DJ,대전,대전


In [12]:
tbl_order_path = tbl_order_paths[0]
print(f'데이터 로딩: {tbl_order_path}')
order_data = pd.read_csv(tbl_order_path)
print(f'데이터 건수: {len(order_data)}')
order_data.head(3)

데이터 로딩: C:/Users/manci/Desktop/data/PythonML-main/PythonML-main/chapter06/data\0_input\tbl_order_202004.csv
데이터 건수: 233374


,order_id,store_id,customer_id,coupon_cd,sales_detail_id,order_accept_date,delivered_date,takeout_flag,total_amount,status
0,6485764,126,C33734772,84,13294339,2020-04-01 11:00,2020-04-01 11:43,0,24710,9
1,89788069,15,C14950605,39,18972168,2020-04-01 11:00,2020-04-01 11:30,0,46920,2
2,67719661,58,C91671073,19,45126916,2020-04-01 11:00,2020-04-01 11:11,0,44620,2


In [14]:
order_data = order_data.loc[order_data['store_id'] != 990]

order_data = pd.merge(order_data, m_store, on = 'store_id', how = 'left')
order_data = pd.merge(order_data, m_area, on = 'area_cd', how = 'left')

order_data.loc[order_data['takeout_flag'] == 0, 'takeout_name'] = 'delivery'
order_data.loc[order_data['takeout_flag'] == 1, 'takeout_name'] = 'takeout'

In [15]:
order_data.loc[order_data['status'] == 0, 'status_name'] = '주문 접수'
order_data.loc[order_data['status'] == 1, 'status_name'] = '계산 완료'
order_data.loc[order_data['status'] == 2, 'status_name'] = '배달 완료'
order_data.loc[order_data['status'] == 9, 'status_name'] = '주문 취소'
order_data.head(3)

,order_id,store_id,customer_id,coupon_cd,sales_detail_id,order_accept_date,delivered_date,takeout_flag,total_amount,status,store_name,area_cd,wide_area,narrow_area,takeout_name,status_name
0,6485764,126,C33734772,84,13294339,2020-04-01 11:00,2020-04-01 11:43,0,24710,9,하신번영로점,BS,부산,부산,delivery,주문 취소
1,89788069,15,C14950605,39,18972168,2020-04-01 11:00,2020-04-01 11:30,0,46920,2,학동로점,SL,서울,서울,delivery,배달 완료
2,67719661,58,C91671073,19,45126916,2020-04-01 11:00,2020-04-01 11:11,0,44620,2,관악로2점,SL,서울,서울,delivery,배달 완료


In [16]:
print(len(order_data))

233374


In [17]:
order_data.isna().sum()

order_id             0
store_id             0
customer_id          0
coupon_cd            0
sales_detail_id      0
order_accept_date    0
delivered_date       0
takeout_flag         0
total_amount         0
status               0
store_name           0
area_cd              0
wide_area            0
narrow_area          0
takeout_name         0
status_name          0
dtype: int64

In [19]:
# 머신러닝 변수 만들기
def calc_delta(t):
    t1, t2, = t
    delta = t2 - t1
    return delta.total_seconds() / 60

order_data.loc[:, 'order_accept_datetime'] = pd.to_datetime(order_data['order_accept_date'])
order_data.loc[:, 'delivered_datetime'] = pd.to_datetime(order_data['delivered_date'])
order_data.loc[:, 'delta'] = order_data[['order_accept_datetime', 'delivered_datetime']].apply(calc_delta, axis = 1)
order_data.head(3)

,order_id,store_id,customer_id,coupon_cd,sales_detail_id,order_accept_date,delivered_date,takeout_flag,total_amount,status,store_name,area_cd,wide_area,narrow_area,takeout_name,status_name,order_accept_datetime,delivered_datetime,delta
0,6485764,126,C33734772,84,13294339,2020-04-01 11:00,2020-04-01 11:43,0,24710,9,하신번영로점,BS,부산,부산,delivery,주문 취소,2020-04-01 11:00:00,2020-04-01 11:43:00,43.0
1,89788069,15,C14950605,39,18972168,2020-04-01 11:00,2020-04-01 11:30,0,46920,2,학동로점,SL,서울,서울,delivery,배달 완료,2020-04-01 11:00:00,2020-04-01 11:30:00,30.0
2,67719661,58,C91671073,19,45126916,2020-04-01 11:00,2020-04-01 11:11,0,44620,2,관악로2점,SL,서울,서울,delivery,배달 완료,2020-04-01 11:00:00,2020-04-01 11:11:00,11.0


In [49]:
order_data.loc[:, 'order_accept_hour'] = order_data['order_accept_datetime'].dt.hour
order_data.loc[:, 'order_accept_weekday'] = order_data['order_accept_datetime'].dt.weekday

order_data.loc[order_data['order_accept_weekday'] >= 5, 'weekday_info'] = '휴일'
order_data.loc[order_data['order_accept_weekday'] < 5, 'weekday_info'] = '평일'

order_data.head(3)

,order_id,store_id,customer_id,coupon_cd,sales_detail_id,order_accept_date,delivered_date,takeout_flag,total_amount,status,...,wide_area,narrow_area,takeout_name,status_name,order_accept_datetime,delivered_datetime,delta,order_accept_hour,order_accept_weekday,weekday_info
0,6485764,126,C33734772,84,13294339,2020-04-01 11:00,2020-04-01 11:43,0,24710,9,...,부산,부산,delivery,주문 취소,2020-04-01 11:00:00,2020-04-01 11:43:00,43.0,11,2,평일
1,89788069,15,C14950605,39,18972168,2020-04-01 11:00,2020-04-01 11:30,0,46920,2,...,서울,서울,delivery,배달 완료,2020-04-01 11:00:00,2020-04-01 11:30:00,30.0,11,2,평일
2,67719661,58,C91671073,19,45126916,2020-04-01 11:00,2020-04-01 11:11,0,44620,2,...,서울,서울,delivery,배달 완료,2020-04-01 11:00:00,2020-04-01 11:11:00,11.0,11,2,평일


In [50]:
# 매장단위 집계(매장별 주문 건수)
store_data = order_data.groupby(['store_name']).count()[['order_id']]
store_f = order_data.loc[
    (order_data['status_name'] == '배달 완료') | 
    (order_data['status_name'] == '결제 완료')
    ].groupby(['store_name']).count()[['order_id']]
store_c = order_data.loc[
    order_data['status_name'] == '주문 취소'
    ].groupby(['store_name']).count()[['order_id']]
store_d = order_data.loc[
    order_data['takeout_name'] == 'delivery'
    ].groupby(['store_name']).count()[['order_id']]
store_t = order_data.loc[
    order_data['takeout_name'] == 'takeout'
    ].groupby(['store_name']).count()[['order_id']]

store_weekday = order_data.loc[
    order_data['weekday_info'] == '평일'
    ].groupby(['store_name']).count()[['order_id']]
store_weekend = order_data.loc[
    order_data['weekday_info'] == '휴일'
    ].groupby(['store_name']).count()[['order_id']]

In [51]:
store_data.head(3)

,order_id
store_name,
가덕해안로점,1000
가마산로2점,1470
가마산로점,1021


In [52]:
# 시간별 주문건수
times = order_data['order_accept_hour'].unique()
store_time = []
for time in times:
    time_tmp = order_data.loc[
        order_data['order_accept_hour'] == time
        ].groupby(['store_name']).count()[['order_id']]
    time_tmp.columns = [f'order_time_{time}']
    store_time.append(time_tmp)

store_time = pd.concat(store_time, axis = 1)
store_time.head(3)

,order_time_11,order_time_12,order_time_13,order_time_14,order_time_15,order_time_16,order_time_17,order_time_18,order_time_19,order_time_20,order_time_21
store_name,,,,,,,,,,,
가덕해안로점,78,118,84,99,66,77,107,91,100,86,94
가마산로2점,181,141,128,112,123,146,132,143,118,121,125
가마산로점,81,105,101,94,93,111,77,81,104,88,86


In [53]:
# 배달 시간 집계와 집계 결과 집합
store_delta = order_data.loc[
    order_data['status_name'] != '주문 취소'
    ].groupby(['store_name']).mean()[['delta']]
store_data.columns = ['order']
store_f.columns = ['order_fin']
store_c.columns = ['order_cancel']
store_d.columns = ['order_delivery']
store_t.columns = ['order_takeout']

store_weekday.columns = ['order_weekday']
store_weekend.columns = ['order_weekend']
store_delta.columns = ['delta_avg']
store_data = pd.concat([
    store_data, store_f, store_c, store_d, store_t,
    store_weekday, store_weekend, store_time, store_delta], axis = 1)
store_data.head(3)

,order,order_fin,order_cancel,order_delivery,order_takeout,order_weekday,order_weekend,order_time_11,order_time_12,order_time_13,order_time_14,order_time_15,order_time_16,order_time_17,order_time_18,order_time_19,order_time_20,order_time_21,delta_avg
store_name,,,,,,,,,,,,,,,,,,,
가덕해안로점,1000,610,182,741,259,732,268,78,118,84,99,66,77,107,91,100,86,94,33.943765
가마산로2점,1470,916,263,1099,371,1076,394,181,141,128,112,123,146,132,143,118,121,125,34.702568
가마산로점,1021,653,183,792,229,748,273,81,105,101,94,93,111,77,81,104,88,86,34.730310


In [54]:
# 데이터 가공 및 매장별 집계 함수로 실행
def data_processing(order_data):
    order_data = order_data.loc[order_data['store_id'] != 990]
    order_data = pd.merge(order_data, m_store, on = 'store_id', how = 'left')
    order_data = pd.merge(order_data, m_area, on = 'area_cd', how = 'left')
    order_data.loc[order_data['takeout_flag'] == 0, 'takeout_name'] = 'delivery'
    order_data.loc[order_data['takeout_flag'] == 1, 'takeout_name'] = 'takeout'
    order_data.loc[order_data['status'] == 0, 'status_name'] = '주문 접수'
    order_data.loc[order_data['status'] == 1, 'status_name'] = '계산 완료'
    order_data.loc[order_data['status'] == 2, 'status_name'] = '배달 완료'
    order_data.loc[order_data['status'] == 9, 'status_name'] = '주문 취소'
    
    order_data.loc[:, 'order_accept_datetime'] = pd.to_datetime(order_data['order_accept_date'])
    order_data.loc[:, 'delivered_datetime'] = pd.to_datetime(order_data['delivered_date'])
    order_data.loc[:, 'delta'] = order_data[['order_accept_datetime', 'delivered_datetime']].apply(calc_delta, axis = 1)
    
    order_data.loc[:, 'order_accept_hour'] = order_data['order_accept_datetime'].dt.hour
    order_data.loc[:, 'order_accept_weekday'] = order_data['order_accept_datetime'].dt.weekday
    order_data.loc[order_data['order_accept_weekday'] >= 5, 'weekday_info'] = '휴일'
    order_data.loc[order_data['order_accept_weekday'] < 5, 'weekday_info'] = '평일'
    
    store_data = order_data.groupby(['store_name']).count()[['order_id']]
    store_f = order_data.loc[
        (order_data['status_name'] == '배달 완료') | 
        (order_data['status_name'] == '결제 완료')
        ].groupby(['store_name']).count()[['order_id']]
    store_c = order_data.loc[
        order_data['status_name'] == '주문 취소'
        ].groupby(['store_name']).count()[['order_id']]
    store_d = order_data.loc[
        order_data['takeout_name'] == 'delivery'
        ].groupby(['store_name']).count()[['order_id']]
    store_t = order_data.loc[
        order_data['takeout_name'] == 'takeout'
        ].groupby(['store_name']).count()[['order_id']]

    store_weekday = order_data.loc[
        order_data['weekday_info'] == '평일'
        ].groupby(['store_name']).count()[['order_id']]
    store_weekend = order_data.loc[
        order_data['weekday_info'] == '휴일'
        ].groupby(['store_name']).count()[['order_id']]
    
    times = order_data['order_accept_hour'].unique()
    store_time = []
    for time in times:
        time_tmp = order_data.loc[
            order_data['order_accept_hour'] == time
            ].groupby(['store_name']).count()[['order_id']]
        time_tmp.columns = [f'order_time_{time}']
        store_time.append(time_tmp)
    store_time = pd.concat(store_time, axis = 1)
    
    store_delta = order_data.loc[
        order_data['status_name'] != '주문 취소'
        ].groupby(['store_name']).mean()[['delta']]
    store_data.columns = ['order']
    store_f.columns = ['order_fin']
    store_c.columns = ['order_cancel']
    store_d.columns = ['order_delivery']
    store_t.columns = ['order_takeout']

    store_weekday.columns = ['order_weekday']
    store_weekend.columns = ['order_weekend']
    store_delta.columns = ['delta_avg']
    store_data = pd.concat([
        store_data, store_f, store_c, store_d, store_t,
        store_weekday, store_weekend, store_time, store_delta], axis = 1)
    
    return store_data   


In [56]:
tbl_order_path = tbl_order_paths[1]
print(f'데이터 로딩: {tbl_order_path}')
order_data = pd.read_csv(tbl_order_path)
store_data = data_processing(order_data)
store_data.head(3)

데이터 로딩: C:/Users/manci/Desktop/data/PythonML-main/PythonML-main/chapter06/data\0_input\tbl_order_202005.csv


,order,order_fin,order_cancel,order_delivery,order_takeout,order_weekday,order_weekend,order_time_11,order_time_12,order_time_13,order_time_14,order_time_15,order_time_16,order_time_17,order_time_18,order_time_19,order_time_20,order_time_21,delta_avg
store_name,,,,,,,,,,,,,,,,,,,
가덕해안로점,1033,634,188,756,277,700,333,86,96,94,99,78,92,88,98,119,100,83,34.269822
가마산로2점,1528,936,276,1128,400,1037,491,133,136,138,135,145,147,147,147,120,141,139,35.023962
가마산로점,1066,640,216,795,271,724,342,93,89,105,99,88,102,94,102,91,105,98,34.463529


In [58]:
# 모든 데이터 로딩 및 데이터 가공
store_all = []
for tbl_order_path in tbl_order_paths:
    print(f'데이터 로딩: {tbl_order_path}')
    tg_ym = tbl_order_path.split('_')[-1][:6]
    order_data = pd.read_csv(tbl_order_path)
    store_data = data_processing(order_data)
    store_data.loc[:, 'year_month'] = tg_ym
    store_data.reset_index(drop = False, inplace = True)
    store_all.append(store_data)
    
store_all = pd.concat(store_all, ignore_index = True)
display(store_all.head(3))
display(store_all.tail(3))
store_monthly_name = 'store_monthly_data.csv'
store_all.to_csv(os.path.join(output_dir, store_monthly_name), index = False)

데이터 로딩: C:/Users/manci/Desktop/data/PythonML-main/PythonML-main/chapter06/data\0_input\tbl_order_202004.csv
데이터 로딩: C:/Users/manci/Desktop/data/PythonML-main/PythonML-main/chapter06/data\0_input\tbl_order_202005.csv
데이터 로딩: C:/Users/manci/Desktop/data/PythonML-main/PythonML-main/chapter06/data\0_input\tbl_order_202006.csv
데이터 로딩: C:/Users/manci/Desktop/data/PythonML-main/PythonML-main/chapter06/data\0_input\tbl_order_202007.csv
데이터 로딩: C:/Users/manci/Desktop/data/PythonML-main/PythonML-main/chapter06/data\0_input\tbl_order_202008.csv
데이터 로딩: C:/Users/manci/Desktop/data/PythonML-main/PythonML-main/chapter06/data\0_input\tbl_order_202009.csv
데이터 로딩: C:/Users/manci/Desktop/data/PythonML-main/PythonML-main/chapter06/data\0_input\tbl_order_202010.csv
데이터 로딩: C:/Users/manci/Desktop/data/PythonML-main/PythonML-main/chapter06/data\0_input\tbl_order_202011.csv
데이터 로딩: C:/Users/manci/Desktop/data/PythonML-main/PythonML-main/chapter06/data\0_input\tbl_order_202012.csv
데이터 로딩: C:/Users/manci/Deskt

,store_name,order,order_fin,order_cancel,order_delivery,order_takeout,order_weekday,order_weekend,order_time_11,order_time_12,...,order_time_14,order_time_15,order_time_16,order_time_17,order_time_18,order_time_19,order_time_20,order_time_21,delta_avg,year_month
0,가덕해안로점,1000,610,182,741,259,732,268,78,118,...,99,66,77,107,91,100,86,94,33.943765,202004
1,가마산로2점,1470,916,263,1099,371,1076,394,181,141,...,112,123,146,132,143,118,121,125,34.702568,202004
2,가마산로점,1021,653,183,792,229,748,273,81,105,...,94,93,111,77,81,104,88,86,34.730310,202004


,store_name,order,order_fin,order_cancel,order_delivery,order_takeout,order_weekday,order_weekend,order_time_11,order_time_12,...,order_time_14,order_time_15,order_time_16,order_time_17,order_time_18,order_time_19,order_time_20,order_time_21,delta_avg,year_month
2325,화중로점,1187,711,234,869,318,882,305,102,123,...,140,107,94,116,107,107,106,95,34.453305,202103
2326,효덕로점,1331,841,234,1018,313,990,341,129,128,...,105,124,105,112,116,133,137,122,34.677302,202103
2327,효원로점,814,491,153,594,220,607,207,82,80,...,65,57,72,85,78,61,75,84,34.998487,202103


In [60]:
y = store_all[
    ['store_name', 'year_month', 'order_weekday', 'order_weekend']].copy()
y.loc[:, 'one_month_ago'] = pd.to_datetime(y['year_month'], format = '%Y%m')

from dateutil.relativedelta import relativedelta

y.loc[:, 'one_month_ago'] = y['one_month_ago'].map(lambda x: x - relativedelta(months = 1))
y.loc[:, 'one_month_ago'] = y['one_month_ago'].dt.strftime('%Y%m')
y.head(3)

,store_name,year_month,order_weekday,order_weekend,one_month_ago
0,가덕해안로점,202004,732,268,202003
1,가마산로2점,202004,1076,394,202003
2,가마산로점,202004,748,273,202003


In [61]:
y_one_month_ago = y.copy()
y_one_month_ago.rename(columns = {
    'order_weekday': 'order_weekday_one_month_ago',
    'order_weekend': 'order_weekend_one_month_ago',
    'year_month': 'year_month_for_join'}, inplace = True)
y = pd.merge(y, y_one_month_ago[['store_name', 'year_month_for_join', 
                                 'order_weekday_one_month_ago', 
                                 'order_weekend_one_month_ago']], 
             left_on = ['store_name', 'one_month_ago'],
             right_on = ['store_name', 'year_month_for_join'], how = 'left')
y.loc[y['store_name'] == '가덕해안로점']

,store_name,year_month,order_weekday,order_weekend,one_month_ago,year_month_for_join,order_weekday_one_month_ago,order_weekend_one_month_ago
0,가덕해안로점,202004,732,268,202003,NaN,NaN,NaN
194,가덕해안로점,202005,700,333,202004,202004,732.0,268.0
388,가덕해안로점,202006,734,262,202005,202005,700.0,333.0
582,가덕해안로점,202007,772,266,202006,202006,734.0,262.0
776,가덕해안로점,202008,701,333,202007,202007,772.0,266.0
970,가덕해안로점,202009,739,267,202008,202008,701.0,333.0
1164,가덕해안로점,202010,729,299,202009,202009,739.0,267.0
1358,가덕해안로점,202011,698,303,202010,202010,729.0,299.0
1552,가덕해안로점,202012,770,266,202011,202011,698.0,303.0
1746,가덕해안로점,202101,705,335,202012,202012,770.0,266.0


In [62]:
y.dropna(inplace = True)
y.loc[y['order_weekday'] - y['order_weekday_one_month_ago'] > 0, 'y_weekday'] = 1
y.loc[y['order_weekday'] - y['order_weekday_one_month_ago'] <= 0, 'y_weekday'] = 0
y.loc[y['order_weekend'] - y['order_weekend_one_month_ago'] > 0, 'y_weekend'] = 1
y.loc[y['order_weekend'] - y['order_weekend_one_month_ago'] <= 0, 'y_weekend'] = 0

In [63]:
# 머신러닝용 데이터 완성
y.rename(columns = {'year_month': 'target_year_month'}, inplace = True)
y = y[['store_name', 'target_year_month', 'one_month_ago', 'y_weekday', 'y_weekend']].copy()
ml_data = pd.merge(y, store_all, left_on = ['store_name', 'one_month_ago'], 
                                 right_on = ['store_name', 'year_month'], how = 'left')
ml_data.head()

,store_name,target_year_month,one_month_ago,y_weekday,y_weekend,order,order_fin,order_cancel,order_delivery,order_takeout,...,order_time_14,order_time_15,order_time_16,order_time_17,order_time_18,order_time_19,order_time_20,order_time_21,delta_avg,year_month
0,가덕해안로점,202005,202004,0.0,1.0,1000,610,182,741,259,...,99,66,77,107,91,100,86,94,33.943765,202004
1,가마산로2점,202005,202004,0.0,1.0,1470,916,263,1099,371,...,112,123,146,132,143,118,121,125,34.702568,202004
2,가마산로점,202005,202004,0.0,1.0,1021,653,183,792,229,...,94,93,111,77,81,104,88,86,34.730310,202004
3,감천항로점,202005,202004,0.0,1.0,1005,589,197,716,289,...,86,99,84,85,91,83,101,97,34.295792,202004
4,강남대로2점,202005,202004,0.0,1.0,1098,651,212,796,302,...,93,110,98,105,101,83,101,103,35.101580,202004


In [64]:
del ml_data['target_year_month']
del ml_data['one_month_ago']
ml_data.head()

,store_name,y_weekday,y_weekend,order,order_fin,order_cancel,order_delivery,order_takeout,order_weekday,order_weekend,...,order_time_14,order_time_15,order_time_16,order_time_17,order_time_18,order_time_19,order_time_20,order_time_21,delta_avg,year_month
0,가덕해안로점,0.0,1.0,1000,610,182,741,259,732,268,...,99,66,77,107,91,100,86,94,33.943765,202004
1,가마산로2점,0.0,1.0,1470,916,263,1099,371,1076,394,...,112,123,146,132,143,118,121,125,34.702568,202004
2,가마산로점,0.0,1.0,1021,653,183,792,229,748,273,...,94,93,111,77,81,104,88,86,34.730310,202004
3,감천항로점,0.0,1.0,1005,589,197,716,289,739,266,...,86,99,84,85,91,83,101,97,34.295792,202004
4,강남대로2점,0.0,1.0,1098,651,212,796,302,805,293,...,93,110,98,105,101,83,101,103,35.101580,202004


In [65]:
ml_data.isna().sum()

store_name        0
y_weekday         0
y_weekend         0
order             0
order_fin         0
order_cancel      0
order_delivery    0
order_takeout     0
order_weekday     0
order_weekend     0
order_time_11     0
order_time_12     0
order_time_13     0
order_time_14     0
order_time_15     0
order_time_16     0
order_time_17     0
order_time_18     0
order_time_19     0
order_time_20     0
order_time_21     0
delta_avg         0
year_month        0
dtype: int64

In [69]:
display(ml_data.groupby('y_weekday').count()[['store_name']])
display(ml_data.groupby('y_weekend').count()[['store_name']])

,store_name
y_weekday,
0.0,1059
1.0,1075


,store_name
y_weekend,
0.0,1118
1.0,1016


In [70]:
ml_data.to_csv(os.path.join(output_dir, 'ml_base_data.csv'), index = False)